#  IRIS Flower classification
The data set from the kaggle:https://www.kaggle.com/datasets/arshid/iris-flower-dataset
### imports
-here we import torch for gradients and networks<br>
-pandas for data processing<br>
-sci-kit learn for its usefull functions<br>
-shuffle for to shuffle the data 

In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

### Reading csv file
* next we will drop the features not relevant to the output, and the label itself
* storing the unique values as the classes
* printing the distribution of the classes
* converting the categorail fields into 1 or 0's 
* finally popping out the label

In [2]:
data = pd.read_csv("./iris.csv")
data = data.drop(["Id"], axis=1)

un_labels = data["Species"].unique()
# for i in data["Species"].unique():
#     print(sum(data["Species"]==i))

data = shuffle(data)
labels = data.pop("Species")
labels = np.array(labels)

data = ss().fit_transform(data)
print(un_labels)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


* next we are creating a zeros array of shape of length of labels and unique labels those are one hot labels
* next where the labels is equal to the unique labels ,there we are equaling to 1
* printing first 10 one hot labels

In [3]:
ohe_labels = np.zeros((len(labels), len(un_labels)))

for i in range(len(un_labels)):
    x = np.where(labels == un_labels[i])
    ohe_labels[x,i] = 1

ohe_labels[:10]

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

## conertions
*  converting the arrays into tensors so we can perform ml operations on them

In [4]:
features = torch.tensor(np.array(data), dtype=torch.float32)
labels = torch.tensor(ohe_labels, dtype=torch.float32)

* creating a reasonally layered model with forst input size as 4 and ending output value as 3 followed by a softmax to convert the value predictions into probablity distributions



In [5]:
model = torch.nn.Sequential(
    torch.nn.Linear(4,4),
    torch.nn.ReLU(),
    torch.nn.Linear(4,3),
    torch.nn.Softmax(dim=-1)
)

preds = model(features)
print(preds[:5])

tensor([[0.1999, 0.4581, 0.3420],
        [0.1908, 0.4718, 0.3374],
        [0.1999, 0.4581, 0.3420],
        [0.2101, 0.4574, 0.3325],
        [0.1930, 0.4684, 0.3385]], grad_fn=<SliceBackward0>)


#####  the loss we will use is Binary cross entropy


In [6]:
loss_fn = torch.nn.BCELoss()

loss = loss_fn(preds, labels)
print(loss)

tensor(0.6696, grad_fn=<BinaryCrossEntropyBackward0>)



##### the optimizer we will use is SGD-(stochastic gradient descent) to update network weights during training

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

#### Here Saving the weights and biases of our model so we can load em up again whenever we want to make a prediction.

In [ ]:
torch.save(model.state_dict(),"savedweights.pth")

#### this is the training loop
* we simply make predictions and calculate loss
* then we will find gradients and optimize the model to reduce loss
    
* next  we are calculating the accuracy of our model
    
* Doing further action only on the 1/tengths of the total epochs to save time
* here we will chech if at the index of the max pred is there a 1 present in the labels<br>  -finally calculating and printing accuracy

In [8]:
epochs = 100000
for epoch in range(epochs):
    preds = model(features)

    loss = loss_fn(preds, labels)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            preds=model(features)

            for i in range(len(preds)):
                ele_index=torch.where(preds[i]==preds[i].max())
                if ohe_labels[i][ele_index]==1:
                    right+=1   

            
            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ len(preds), 2)}%")       

Loss : 0.28520962595939636
Accuracy : 83.33%
Loss : 0.19271308183670044
Accuracy : 91.33%
Loss : 0.14275170862674713
Accuracy : 95.33%
Loss : 0.10984338819980621
Accuracy : 96.67%
Loss : 0.08840013295412064
Accuracy : 97.33%
Loss : 0.07460162788629532
Accuracy : 97.33%
Loss : 0.06548945605754852
Accuracy : 97.33%
Loss : 0.059073012322187424
Accuracy : 97.33%
Loss : 0.05438174679875374
Accuracy : 97.33%
Loss : 0.050838399678468704
Accuracy : 97.33%
